In [1]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import pickle


import math
print('TF version',tf.__version__)

TF version 2.1.0


In [2]:
MAX_LEN = 96
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file='../input/tf-roberta/vocab-roberta-base.json', 
    merges_file='../input/tf-roberta/merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
EPOCHS = 3 # originally 3
BATCH_SIZE = 32 # originally 32
PAD_ID = 1
#SEED = 88888
LABEL_SMOOTHING = 0.1
#tf.random.set_seed(SEED)
#np.random.seed(SEED)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('../input//tweet-sentiment-extraction/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
input_ids_ = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
attention_mask_ = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
sentiments =  np.zeros((ct ,3))
for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
    if train.loc[k ,'sentiment'] == 'positive':
        sentiments[k][0] = 1
    elif train.loc[k ,'sentiment'] == 'negative':
        sentiments[k][1] = 1
    elif train.loc[k ,'sentiment'] == 'neutral':
        sentiments[k][2] = 1
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0, s_tok ,2 ,2] + enc.ids + [2]
    input_ids_[k,:len(enc.ids)+2] = [0] + enc.ids + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    attention_mask_[k,:len(enc.ids)+2] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+4] = 1
        end_tokens[k,toks[-1]+4] = 1
    if k == 2:
        print(start_tokens[k])
        print(end_tokens[k])
        print(text1)
        print(input_ids[k])
        print(text2)
        a = np.argmax(start_tokens[k])
        b = np.argmax(end_tokens[k])
        man = tokenizer.encode(text1)
        print(tokenizer.decode(enc.ids[a-4:b-3]))

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 my boss is bullying me...
[    0  2430     2     2   127  3504    16 11902   162   734     2     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1    

In [4]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
input_ids_t_ = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
attention_mask_t_ = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0, s_tok ,2 ,2] + enc.ids + [2]
    input_ids_t_[k,:len(enc.ids)+2] = [0] + enc.ids + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1
    attention_mask_t_[k,:len(enc.ids)+2] = 1

In [5]:
config = RobertaConfig.from_pretrained('../input/tf-roberta/config-roberta-base.json')
bert_model = TFRobertaModel.from_pretrained('../input/tf-roberta/pretrained-roberta-base.h5',config=config)

In [6]:
def save_weights(model, dst_fn):
    weights = model.get_weights()
    with open(dst_fn, 'wb') as f:
        pickle.dump(weights, f)


def load_weights(model, weight_fn):
    with open(weight_fn, 'rb') as f:
        weights = pickle.load(f)
    model.set_weights(weights)
    return model

def loss_fn(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss
def scheduler(epoch):
    return 3e-5 * 0.2**epoch

def build_model():
    config = RobertaConfig.from_pretrained('../input/tf-roberta/config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained('../input/tf-roberta/pretrained-roberta-base.h5',config=config)
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    ids1 = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att1 = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    padding = tf.cast(tf.equal(ids, PAD_ID), tf.int32)
    
    lens = MAX_LEN - tf.reduce_sum(padding, -1)
    max_len = tf.reduce_max(lens)
    ids_ = ids[:, :max_len]
    att_ = att[:, :max_len]
    tok_ = tok[:, :max_len]
    x = bert_model(ids_,attention_mask=att_,token_type_ids=tok_)
    x_ = bert_model(ids1,attention_mask=att1)
    x1 = tf.keras.layers.Dropout(0.4)(x[0])
    x1 = tf.keras.layers.Conv1D(1024, 2,padding='same')(x1)
    x1_ = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dropout(0.4)(x1_)
    x1 = tf.keras.layers.Conv1D(768, 2,padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.4)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1024, 2,padding='same')(x2)
    x2_ = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dropout(0.4)(x2_)
    x2 = tf.keras.layers.Conv1D(768, 2,padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)
    x_ = tf.keras.layers.Conv1D(1024, 2,padding='same')(x_[0])
    x_ = tf.keras.layers.Dense(1)(x_)
    x_ = tf.keras.layers.LeakyReLU()(x_)
    x_ = tf.keras.layers.Flatten()(x_)
    x_ = tf.keras.layers.Dense(3 ,activation = 'softmax')(x_)
    model = tf.keras.models.Model(inputs=[ids,ids1 ,att ,att1 ,tok], outputs=[x1,x2 ,x_])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5) 
    model.compile(loss=loss_fn, optimizer=optimizer)
    
    # this is required as `model.predict` needs a fixed size!
    x1_padded = tf.pad(x1, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    x2_padded = tf.pad(x2, [[0, 0], [0, MAX_LEN - max_len]], constant_values=0.)
    padded_model = tf.keras.models.Model(inputs=[ids , att, tok], outputs=[x1_padded,x2_padded])
    return model, padded_model

In [7]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [8]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True) #,random_state=SEED) #originally 5 splits
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model, padded_model = build_model()
        
    #sv = tf.keras.callbacks.ModelCheckpoint(
    #    '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
    #    save_weights_only=True, mode='auto', save_freq='epoch')
    inpT = [input_ids[idxT,],input_ids_[idxT,] ,attention_mask[idxT,],attention_mask_[idxT,], token_type_ids[idxT,]]
    targetT = [start_tokens[idxT,], end_tokens[idxT,] ,sentiments[idxT ,]]
    inpV = [input_ids[idxV,],input_ids_[idxV,],attention_mask[idxV,],attention_mask_[idxV,] ,token_type_ids[idxV,]]
    targetV = [start_tokens[idxV,], end_tokens[idxV,] ,sentiments[idxT ,]]
    # sort the validation data
    shuffleV = np.int32(sorted(range(len(inpV[0])), key=lambda k: (inpV[0][k] == PAD_ID).sum(), reverse=True))
    inpV = [arr[shuffleV] for arr in inpV]
    targetV = [arr[shuffleV] for arr in targetV]
    weight_fn = '%s-roberta-%i.h5'%(VER,fold)
    for epoch in range(1, EPOCHS + 1):
        # sort and shuffle: We add random numbers to not have the same order in each epoch
        shuffleT = np.int32(sorted(range(len(inpT[0])), key=lambda k: (inpT[0][k] == PAD_ID).sum() + np.random.randint(-3, 3), reverse=True))
        # shuffle in batches, otherwise short batches will always come in the beginning of each epoch
        num_batches = math.ceil(len(shuffleT) / BATCH_SIZE)
        batch_inds = np.random.permutation(num_batches)
        shuffleT_ = []
        for batch_ind in batch_inds:
            shuffleT_.append(shuffleT[batch_ind * BATCH_SIZE: (batch_ind + 1) * BATCH_SIZE])
        shuffleT = np.concatenate(shuffleT_)
        # reorder the input data
        inpT = [arr[shuffleT] for arr in inpT]
        targetT = [arr[shuffleT] for arr in targetT]
        reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
        model.fit(inpT, targetT, 
            epochs=epoch, initial_epoch=epoch - 1, batch_size=BATCH_SIZE, verbose=DISPLAY, callbacks=[reduce_lr],
            validation_data=(inpV, targetV), shuffle=False)  # don't shuffle in `fit`
        save_weights(model, weight_fn)

    print('Loading model...')
    # model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    load_weights(model, weight_fn)

    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = padded_model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = padded_model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-4:b-3])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################
Train on 21984 samples, validate on 5497 samples
21984/21984 [==============================] - 413s 19ms/sample - loss: 3.9535 - activation_loss: 1.5709 - activation_1_loss: 1.5711 - dense_3_loss: 0.8114 - val_loss: 4.2578 - val_activation_loss: 1.2975 - val_activation_1_loss: 1.2715 - val_dense_3_loss: 1.6892
Train on 21984 samples, validate on 5497 samples
Epoch 2/2
21984/21984 [==============================] - 371s 17ms/sample - loss: 3.2467 - activation_loss: 1.3261 - activation_1_loss: 1.2925 - dense_3_loss: 0.6280 - val_loss: 4.4185 - val_activation_loss: 1.2790 - val_activation_1_loss: 1.2529 - val_dense_3_loss: 1.8871
Train on 21984 samples, validate on 5497 samples
Epoch 3/3
21984/21984 [==============================] - 371s 17ms/sample - loss: 3.1572 - activation_loss: 1.2926 - activation_1_loss: 1.2599 - dense_3_loss: 0.6048 - val_loss: 4.4770 - val_activation_loss: 1.2744 - val_activation_1_loss: 1.2500 - val

In [9]:
print('>>>> OVERALL 5Fold CV Jaccard =',np.mean(jac))

>>>> OVERALL 5Fold CV Jaccard = 0.709906612350373


In [10]:
print(jac) # Jaccard CVs
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-2:b-1])
    all.append(st)

[0.710678326853381, 0.7089116898957064, 0.7099556268441733, 0.7075765109922423, 0.712410907166362]


In [11]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
1253,b09213d54b,no. my school will start on June1. two days to go. i s...,negative,don`t want. :l
3063,9f23e5d8f5,someone doesn`t feel good...,negative,`t feel good...
172,5ff525b74b,Wondering if i cld make things any worse than they alrea...,negative,they
3372,817b6da55c,"Okay, so the only reason I`m not buying this app is beca...",neutral,so the only reason i`m not buying this app is because i...
2777,3593dbe193,is all alone for the evening!,neutral,alone for the evening!
473,b56108fc36,hahaha niene ur soo smart.! lmao i think that is who it...,positive,in.. whts
1404,f61ae5403a,"It`s 12AM! , and I are still up, sending you as much t...",neutral,"s 12am! , and i are still up, sending you as much tweets..."
2689,1b4612b286,I have a job at camp!! Only downfall? No midnight showi...,neutral,a job at camp!! only downfall? no midnight showing of h...
2846,44da94a9b7,Want to hang out with beth and tenaya and their hubbys t...,neutral,hang out with beth and tenaya and their hubbys tonight....
2905,416e6352b9,are you in need of another kiss attack??,neutral,in need of another kiss attack??
